## JUnit
JUnit is the defacto Java unit testing framework. The latest version is 5, however here JUnit 4 is discussed as it is more widely used.  Before actually using JUnit, let us write our own testing logic. We'll be testing the below Calculator class:

In [ ]:
class Calculator {
    public int add(int x, int y){
        return x + y;
    }
    
    public int subtract(int x, int y){
        return x - y;
    }
}

To test the functionality, we can write the following:

In [ ]:
class CalculatorTest {
    public static void main(String... args){
        Calculator calc = new Calculator();
        
        int result = calc.add(5, 6);
        int expected = 11;
        
        if(result != expected){
            System.out.println("Unexpected result " + result);
        }
    }
}

It would be better if we use exception to denote test failure. Meanwhile it would be better if we modularise our code, such that testing other aspects of calculator becomes a bit easier.

In [ ]:
class CalculatorTest {
    public int errorCount = 0;
    
    public void testAdd(){
        Calculator calc = new Calculator();
        
        int result = calc.add(5, 6);
        int expected = 11;
        
        if(result != expected){
            throw new IllegalStateException("Unexpected result " + result);
        }
    }
    
    public static void main(String... args){
        CalculatorTest cTest = new CalculatorTest();
        
        try {
            cTest.testAdd();
        } catch (Exception e){
            cTest.errorCount++;
            e.printStackTrace();
        }
        
        if(cTest.errorCount > 0){
            throw new IllegalStateException("There were " + cTest.errorCount + " errors");
        }
    }
}

Using a testing framework like JUnit removes some of the extra code that we have to write. We would just need to define test case, it is the responsibility of the framework to execute it and present the result. Given that, we need to familiarize with some of the associated terms:
- **Test:** a method annotated with `@Test`
- **Test class:** contains one or many tests
- **Suite:** allows us to group test classes
- **Runner:** class runs test.

The equivalent JUnit test class is:

In [ ]:
import static org.junit.Assert.assertEquals;

import org.junit.Test;

public class CalculatorTest {
    
    @Test
    public void testAdd() {
        Calculator calculator = new Calculator();
        int result = calculator.add(5, 6);
        int expected = 11;

        assertEquals("Unexpected result", result, expected);
    }
}

There are a number of assertions available:

In [ ]:
@Test
public void testAssertions() {
    // Uses equals
    assertEquals("Hello", new String("Hello")); // Pass
    assertEquals("Hello", "Hello");             // Pass

    // Uses ==
    assertSame("Hello", new String("Hello"));   // Fail
    assertSame("Hello", "Hello");               // Pass

    // Tests for content of arrays
    assertArrayEquals(new char[] { 'A', 'B', 'C' }, 
                      "ABC".toCharArray());     // Pass
    assertArrayEquals(new String[] { new String("H"), "E", "O" }, 
                      new String[] { "H", "E", "O" }); // Pass
    
    // True false
    assertTrue(5 == 5);    // Pass
    assertFalse(6 > 5);    // Pass

    assertNull(null);      // Pass
}

There are also a negations of above assertions available such as `assertNotNull` and `assertNotSame`  

If some code needs to be repeated before and after every test execution, we can make use of `@Before` and `@After` annotations:

In [ ]:
import org.junit.After;
import org.junit.Before;
import org.junit.Test;

public class CalculatorTest {
    private Calculator calculator;

    @Before
    public void initCalculator() {
        calculator = new Calculator();
    }

    @After
    public void destroyCalculator() {
        calculator = null;
    }

    @Test
    public void testAdd() {
        int result = calculator.add(5, 6);
        int expected = 11;

        assertEquals("Unexpected result", result, expected);
    }

    @Test
    public void testSubtract() {
        int result = calculator.subtract(5, 6);
        int expected = -1;

        assertEquals("Unexpected result", result, expected);
    }
}

There are also a `@BeforeClass` and `@AfterClass` annotations (applied to public static method). These methods run before and after all tests have completed.  

To test for occurance of exception:

In [ ]:
@Test(expected = ArithmeticException.class)
public void testDivisionByZero() {
    calculator.divide(5, 0);
}

To test for timeout:

In [ ]:
@Test(timeout = 10)
public void testTimeout() {
    // Long running task
}

To exclude a test from execution:

In [ ]:
@Test
@Ignore
public void someTestToBeSkipped(){
    // Implementation
}

**Runner:** JUnit by default uses `BlockJUnit4ClassRunner` to run all tests. We can use `@RunWith` annotation to use a custom runner. The `Suite` runner helps grouping test classes together and running the test suite: 

In [ ]:
import org.junit.runner.RunWith;
import org.junit.runners.Suite;
import org.junit.runners.Suite.SuiteClasses;

@RunWith(Suite.class)
@SuiteClasses(value = { CalculatorTest.class, ConverterTest.class })
public class MachineTestSuite {

}

There is another runner called as parameterised runner. It helps testing for multiple values in a single test.

In [ ]:
import org.junit.Test;
import org.junit.runner.RunWith;
import org.junit.runners.Parameterized;
import org.junit.runners.Parameterized.Parameter;
import org.junit.runners.Parameterized.Parameters;

@RunWith(Parameterized.class)
public class CalculatorTest {

    @Parameter(1)
    public int operand1;
    @Parameter(2)
    public int operand2;
    @Parameter(0)
    public int expected;

    @Parameters(name="{index}: {1}+{2}={0}")  // Name parameter helps identify the test case
    public static Collection<Integer[]> data() {
        return Arrays.asList(new Integer[][] { 
            { 0, 0, 0 }, { 0, 5, -5 }, { 10, 6, 4 }, { -6, -3, -3 } // All the test cases
        });
    }

    @Test
    public void testAdd() {
        Calculator calculator = new Calculator();
        
        int result = calculator.add(operand1, operand2);
        assertEquals("Unexpected result", result, expected);
    }
}

### Assertions
JUnit comes with `Assert` class which contains several static assert methods such as `assertTrue`, `assertFalse`, `assertEquals`, `assertArrayEquals`, `assertSame`, etc. These methods throw `AssertionError` on failure. These methods also have an overloaded variants which accepts a message passed to the `AssertionError`.

Another way to write assertions is using Hamcrest `assertThat` method in combination with Hamcrest matchers. JUnit itself has a `Assert.assertThat` method which is more or less same as Hamcrest one, but since JUnit 4.13, it has been deprecated. JUnit 4 comes bundled with Hamcrest core 1.3 library. However the latest version is 2.2, so some care must be taken before upgrade as explained here: (Hamcrest distribution)[http://hamcrest.org/JavaHamcrest/distributables] For example:

In [ ]:
Assert.assertThat("hello", equalToIgnoringCase("HELLO"));

Some other examples:

In [ ]:
 List<Integer> nums = new ArrayList<>();
nums.add(5); nums.add(6);
Assert.assertThat(nums, Matchers.hasSize(2));
Assert.assertThat(nums, Matchers.contains(5, 6));

String[] items = {"a", "b", "c"};
Assert.assertThat(items, Matchers.arrayWithSize(3));
Assert.assertThat(items, Matchers.arrayContaining("a", "b", "c"));
Assert.assertThat(items, Matchers.hasItemInArray("a"));

Assert.assertThat(1.2, Matchers.closeTo(1, 0.5));
Assert.assertThat("", Matchers.isEmptyOrNullString());

We can improve readability using `is` method.

In [ ]:
assertThat(" World", is(equalToIgnoringWhiteSpace("World")));

To negate, use `not`

In [ ]:
String str1 = "calligraphy";
String str2 = "call";
assertThat(str1, not(containsString(str2)));

Logical OR equivalent:

In [ ]:
String str = "calligraphy";
String start = "call";
String end = "foo";
assertThat(str, anyOf(startsWith(start), containsString(end)));

Logical AND equivalent:

In [ ]:
String str = "calligraphy";
String start = "call";
String end = "phy";
assertThat(str, allOf(startsWith(start), endsWith(end)));

## Mockito
Mockito mocks not only interfaces but also abstract classes and concrete non-final classes. Out of the box, Mockito cannot mock final classes and final or static methods, but if we really need it, Mockito 2 provides the experimental MockMaker plugin. Methods `equals()` and `hashCode()` cannot be mocked. As an example:

In [ ]:
import static org.mockito.Mockito.mock;
    
    public class PrintService {
        public String print(String message) throws InterruptedException {
            Thread.currentThread().sleep(1000);
            System.out.println("Message: " + message);
            return message;
        }
    }

    PrintService printService = mock(PrintService.class);

    @Test
    public void testPrintMessage() throws InterruptedException {
        String actual = printService.print("Hello World!");
        String expected = "Hello World!";

        Assert.assertEquals(actual, expected); // This assertion will fail, actual is null
    }

By default, all methods of a mock return “uninitialized” or “empty” values, e.g., zeros for numeric types (both primitive and boxed), false for booleans, and nulls for most other types (including String). We want to configure the mock and define what to do when specific methods of the mock are called. This is called stubbing.

In [ ]:
import static org.mockito.Mockito.when;

@Test
public void testPrintMessage() throws InterruptedException {
    // defining what the method does when called
    when(printService.print("Hello World!")).thenReturn("Hello World!");

    String actual = printService.print("Hello World!");
    String expected = "Hello World!";

    Assert.assertEquals(actual, expected);
}

Another way to express the same stubbing is using `doReturn` method:

In [ ]:
doReturn("Hello World!").when(printService).print("Hello World!");

We can also specify multiple values which will be returned as the results of consecutive method calls. The last value will be used as a result for all further method calls.

In [ ]:
when(printService.print("Hello World!")).thenReturn("Hello World!", "Foo Bar");
// or
doReturn("Hello World!", "Foo Bar").when(printService).print("Hello World!");
// or
when(printService.print("Hello World!")).thenReturn("Hello World!").thenReturn("Foo Bar");

To set up a custom answer to be returned when a method is called, we do:

In [ ]:
when(printService.print("World!")).thenAnswer(invocation -> "Hello " + invocation.getArgument(0));
// or        
doAnswer(invocation -> "Hello " + invocation.getArgument(0)).when(printService).print("World!");

This can be used to throw exceptions:

In [ ]:
when(printService.print(null)).thenAnswer(invocation -> {throw new IllegalArgumentException();});
// but prefer the below
when(printService.print(null)).thenThrow(IllegalArgumentException.class);
//or
when(printService.print(null)).thenThrow(new IllegalArgumentException());

If for some reason we want to delegate method call to the actual method:

In [ ]:
Date mock = mock(Date.class);
when(mock.getTime()).thenCallRealMethod();
doCallRealMethod().when(mock).setTime(42);
mock.setTime(42);
assertEquals(42, mock.getTime());

Note that if we create a mock of an interface and try to configure a stub to call a real method, Mockito will throw an exception, as expected. Also, Mockito ensures that the exception being thrown is valid for that specific stubbed method and will complain if the exception is not in the method’s checked exceptions list.

Sometimes we may want to stub a method for a wide range of arguments (instead of a single value as shown earlier). To do this we make use of `ArgumentMatcher`:

In [ ]:
public interface Uppercase {
    boolean close(String input);
}

Uppercase uC = mock(Uppercase.class);
when(uC.close(anyString())).doReturn(true);

Mockito requires you to provide all arguments either by matchers or by exact values. So if a method has more than one argument and we want to use argument matchers for only some of its arguments, it can’t be done. We can’t write code like this:

In [ ]:
// Consider the method
public interface Check {
    public boolean positive(int number, int delay);
}

Check check = mock(Check.class);
when(check.positive(5, anyInt())).doReturn(true);

We need to write it like this:

In [ ]:
public interface Check {
    public boolean positive(int number, int delay);
}

Check check = mock(Check.class);
// eq ArgumentMatcher means specific value
// anyInt means any integer passed
when(check.positive(eq(5), anyInt())).doReturn(true);

Some commonly used `ArgumentMatcher`s include `anyInt`, `anyByte`, `anyCollection`, `anyFloat`, `anyBoolean`, `anyMap`, etc.

### Mockito Annotations
Mockito provides a number of annotations to declaratively supply mock (and other uses), like `@Mock` and `@InjectMocks`. Consider the example below:

In [ ]:
// Need to test Game, however it has a dependency Player
class Game {
    private Player player;

    public Game(Player player) {
        this.player = player;
    }

    public String attack() {
        return "Player attack with: " + player.getWeapon();
    }
}

class Player {
    private String weapon;

    public Player(String weapon) {
        this.weapon = weapon;
    }

    String getWeapon() {
        return weapon;
    }
}

@RunWith(MockitoJUnitRunner.class) // This Runner needs to be used in order to 
                                   // recognize @Mock and @InjectMocks
class GameTest {
    // Using this annotation is similar to player = mock(Player.class)
    @Mock
    Player player;

    // Mockito will put the above created mock Player into Game
    // This doesn't mean Game will be mocked
    @InjectMocks
    Game game;

    @Test
    public void attackWithSwordTest() throws Exception {
        // Defining behavious of the mocked dependency
        Mockito.when(player.getWeapon()).thenReturn("Sword");

        assertEquals("Player attack with: Sword", game.attack());
    }
}